**AI & Machine Learning (KAN-CINTO4003U) - Copenhagen Business School | Spring 2025**

***

# Part II: Agents

Please see the description of the assignment in the README file (section 1) <br>
**Guide notebooks**: [guides/router_agents_guide.ipynb](guides/agents_guide.ipynb) and [guides/tool_agents_guide.ipynb](guides/tool_agents_guide.ipynb)


***
<br>

* Play around with the agent systems in the guides. Maybe try to create a similar system with new nodes? Or you can try to add new tools to the existing tool-using agent system.

* Remember to include some reflections on your results. Are there, for example, any hyperparameters that are particularly important?

* You should follow the steps given in the `router_agents_guide` notebook or the `tool_agents_guide` notebook to create your own agent system - or try to optimize the existing ones.

<br>

***

## Imports

In [1]:
import os
import requests
import streamlit as st
from decouple import config
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from langchain_ibm import ChatWatsonx

## Call the API

In [2]:

WX_API_KEY = config("WX_API_KEY")
WX_PROJECT_ID = config("WX_PROJECT_ID")
WX_API_URL = "https://us-south.ml.cloud.ibm.com"
os.environ["OPENWEATHERMAP_API_KEY"] = config("OPENWEATHERMAP_API_KEY")

## Creating the weather tool

In [3]:
#Here we requested API key for the OpenWeatherMap API
@tool
def get_current_weather(city: str = "Copenhagen", units: str = "metric"):
    """
    Retrieves the current weather data for a specified city.
    
    """
    api_key = os.environ.get("OPENWEATHERMAP_API_KEY")

    response = requests.get(
        "http://api.openweathermap.org/data/2.5/weather",
        params={
            "q": city,
            "appid": api_key,
            "units": units
        }
    )

    data = response.json()

    if response.status_code != 200 or "main" not in data:
        return {
            "error": f"Unable to retrieve weather for {city}",
            "details": data.get("message", "No additional information available.")
        }

    return {
        "city": data.get("name", city),
        "temperature": data["main"]["temp"],
        "weather": data["weather"][0]["main"],
        "humidity": data["main"]["humidity"],
        "wind_speed": data["wind"]["speed"],
        "forecast": data["weather"][0]["description"],
        "icon": data["weather"][0]["icon"],
        "weather_tomorrow": data["main"]["temp_max"],
        "weather_yesterday": data["main"]["temp_min"],
        "rain": data.get("rain", {}).get("1h", 0),
        "snow": data.get("snow", {}).get("1h", 0),
    }



chat_model = ChatWatsonx(
    url=WX_API_URL,
    apikey=WX_API_KEY,
    project_id=WX_PROJECT_ID,
    model_id="mistralai/mistral-large",
)


In [4]:

tools = [get_current_weather]


# We turned the debug flag off to avoid printing the entire conversation in the console.
graph = create_react_agent(chat_model, tools=tools, debug=False)


response = graph.invoke(
    {
        
        "messages": [
            ("user", "Whats the weather in Aarhus?"),
        ]
    },
    debug=False
)

# Print the final AI message
print(response["messages"][-1].content)

 The current weather in Aarhus is 10.37°C with overcast clouds. The humidity is 92% and the wind speed is 1.97 m/s.

The weather tomorrow will be 10.47°C and yesterday it was 9.22°C. There is no rain or snow.


## Tool Agent

For this part of the assignment, we have decided to create a tool agent which tells a weather report based on the location given in the prompt. 

Regarding accuracy, the weather data retrieved is directly from a reliable external source (OpenWeatherMap). We have done some manual checking, and the model was working according to our expectations.

Since this agent relies primarily on a pre-trained language model (Mistral) and simple tool-calling logic rather than traditional ML training, hyperparameter tuning was minimal. We have used minstral large for this task, as it provided quality and reliable responses.

For the weather report we have used mistral-large llm and retreived the weather data using openweathermap's API. To create the weather report, no complex data preprocessing was required, since the weather quries don't need any preprocessing, and the data is straightforward. We didn't feel the need to convert metrics into other formats (Farenheit, knots, etc...), since we are addressing the model on european terms. This could be one extra tool to compliment our solution.

Here's how we would go about checking if the agent is doing a good job:

- Functionality Check: Make sure it actually calls the weather tool when someone asks about the weather.
- Handling Mistakes: Try giving it wrong or weird input (like a fake city) to see if it can handle errors nicely without crashing.
- Ease of Use: Test how flexible it is with different ways of asking the same thing — ideally, users shouldn't need to phrase things super precisely for it to work.